### HUNGARIAN ALGORITHM

In [8]:
from typing import List, Tuple
import math

def hungarian(cost: List[List[float]]) -> Tuple[float, List[int]]:
    """
    Hungarian algorithm finds the cheapest possible match between the rows and columns of a cost matrix.
    Input: cost - n x n matrix
    Output: (min_cost, assignment_row_to_col)
    assignment_row_to_col[i] = j means: row i assigned to column j
    """
    EPS = 1e-12

    n = len(cost)
    if n == 0 or any(len(r) != n for r in cost):
        raise ValueError("The matrix must be square and non-empty.")
    # value validation
    for r in cost:
        for x in r:
            if not isinstance(x, (int, float)) or not math.isfinite(x):
                raise ValueError("All costs must be finite numbers (no NaN/inf).")

    a = [row[:] for row in cost]

    # STEP 1: row reduction
    for i in range(n):
        m = min(a[i])
        for j in range(n):
            a[i][j] -= m

    # STEP 2: reducing columns
    for j in range(n):
        m = min(a[i][j] for i in range(n))
        for i in range(n):
            a[i][j] -= m

    star = [[False] * n for _ in range(n)]           #zero selected for current assignment (part of the solution)
    prime = [[False] * n for _ in range(n)]          #temporary, auxiliary zero
    row_covered = [False] * n                        #coverages tell us which rows/columns we are currently ignoring in the search
    col_covered = [False] * n

    def is_zero(x: float) -> bool:
        return abs(x) < EPS

    def find_zero():
        for i in range(n):
            if row_covered[i]:
                continue
            for j in range(n):
                if not col_covered[j] and is_zero(a[i][j]):
                    return i, j
        return -1, -1

    def find_star_in_row(i):
        for j in range(n):
            if star[i][j]:
                return j
        return -1

    def find_star_in_col(j):
        for i in range(n):
            if star[i][j]:
                return i
        return -1

    def find_prime_in_row(i):
        for j in range(n):
            if prime[i][j]:
                return j
        return -1

    # STEP 3: Starring the Zeros
    for i in range(n):
        for j in range(n):
            if is_zero(a[i][j]) and not row_covered[i] and not col_covered[j]:
                star[i][j] = True
                row_covered[i] = True
                col_covered[j] = True

    row_covered = [False] * n
    col_covered = [False] * n

    def cover_columns_with_stars():
        for j in range(n):
            col_covered[j] = any(star[i][j] for i in range(n))

    cover_columns_with_stars()

    # MAIN LOOP
    while sum(col_covered) < n:
        i, j = find_zero()
        while i == -1:
            # STEP 4: Creating new zeros
            min_uncovered = float("inf")
            for r in range(n):
                if row_covered[r]:
                    continue
                for c in range(n):
                    if not col_covered[c]:
                        min_uncovered = min(min_uncovered, a[r][c])

            if not math.isfinite(min_uncovered):
                raise RuntimeError("Could not find minimum in uncovered elements (data/numerics).")

            for r in range(n):
                for c in range(n):
                    if not row_covered[r] and not col_covered[c]:
                        a[r][c] -= min_uncovered
                    elif row_covered[r] and col_covered[c]:
                        a[r][c] += min_uncovered

            i, j = find_zero()

        prime[i][j] = True
        star_col = find_star_in_row(i)

        if star_col != -1:
            row_covered[i] = True
            col_covered[star_col] = False
        else:
            # STEP 5: Alternating Path
            path = [(i, j)]
            while True:
                r = find_star_in_col(path[-1][1])
                if r == -1:
                    break
                path.append((r, path[-1][1]))
                c = find_prime_in_row(path[-1][0])
                path.append((path[-1][0], c))

            for (r, c) in path:
                star[r][c] = not star[r][c]

            prime = [[False] * n for _ in range(n)]
            row_covered = [False] * n
            col_covered = [False] * n
            cover_columns_with_stars()

    assignment = [-1] * n
    for i in range(n):
        for j in range(n):
            if star[i][j]:
                assignment[i] = j
                break

    if any(x == -1 for x in assignment):
        raise RuntimeError("Full assignment not found (numeric/data problem).")

    min_cost = sum(cost[i][assignment[i]] for i in range(n))
    return min_cost, assignment

In [9]:
if __name__ == "__main__":
    C = [
        [20, 40, 10, 50],
        [100, 80, 30, 40],
        [10, 5, 60, 20],
        [70, 30, 10, 25]
    ]
    val, ass = hungarian(C)
    print("Minimum cost:", val)
    print("Pairs (row, column, cost):", [(i, ass[i], C[i][ass[i]]) for i in range(len(C))])

Minimum cost: 75
Pairs (row, column, cost): [(0, 0, 20), (1, 3, 40), (2, 1, 5), (3, 2, 10)]
